In [1]:
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import mean
from collections import Counter


class PSTool:
    def pyspark_session(self, host_location):
        """
        Creates and returns spark session object
        """
        sc = SparkContext(host_location)  # Create spark context
        spark = SparkSession(sc)  # Create session
        return spark

    def file_loader(self, path, delim, spark_obj):
        data = spark_obj.read.options(delimiter=delim).csv(path)  # load file
        data = data
        return data


if __name__ == "__main__":
    # Instanciate object
    pstool = PSTool()
    # start session
    spk = pstool.pyspark_session('local')
    # load data
    path = 'all_bacilli_subset.tsv'
    df=pstool.file_loader(path, '\t', spk)

22/06/17 16:34:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
# 'Protein _accession', 'Sequence_MD5_digest', 'Sequence_length', 'Analysis',
# 'Signature_accession', 'Signature_description', 'Start location', 'Stop location',
# 'Score', 'Status', 'Date', 'InterPro annotations', 'InterPro annotations' - description (e.g. BRCA2 repeat)
#     (GO annotations (e.g. GO:0005515) - optional column; only displayed if –goterms option is switched on)
#     (Pathways annotations (e.g. REACT_71) - optional column; only displayed if –pathways option is switched on)

The TSV format presents the match data in columns as follows:

    1. Protein accession (e.g. P51587)
    2. Sequence MD5 digest (e.g. 14086411a2cdf1c4cba63020e1622579)
    3. Sequence length (e.g. 3418)
    4. Analysis (e.g. Pfam / PRINTS / Gene3D)
    5. Signature accession (e.g. PF09103 / G3DSA:2.40.50.140)
    6. Signature description (e.g. BRCA2 repeat profile)
    7. Start location
    8. Stop location
    9. Score - is the e-value (or score) of the match reported by member database method (e.g. 3.1E-52)
    10. Status - is the status of the match (T: true)
    11. Date - is the date of the run
    12. InterPro annotations - accession (e.g. IPR002093)
    13. InterPro annotations - description (e.g. BRCA2 repeat)
    14. (GO annotations (e.g. GO:0005515) - optional column; only displayed if –goterms option is switched on)
    15. (Pathways annotations (e.g. REACT_71)-optional column; only displayed if –pathways option is switched on)


# Question 1
How many distinct protein annotations are found in the dataset? I.e. how many distinc InterPRO numbers are there?

In [4]:
InterPRO_distinc = df.select("_c11").distinct().count()
print(InterPRO_distinc)

61


# Question 2
How many annotations does a protein have on average?

In [5]:
total_annot = df.select("_c11").count()
total_annot

200

In [6]:
avg_annot = total_annot/InterPRO_distinc
print(avg_annot)

3.278688524590164


# Question 3
What is the most common GO Term found?

In [7]:
# df.groupBy("_c13").count().show()

In [16]:
go_list = []
for i in df.filter(df._c13 != '-').select('_c13').collect():
    go_list.extend(i[0].split("|"))

In [18]:
most_common = max(go_list, key = go_list.count)
most_common

'GO:0004743'

# Question 4
What is the average size of an InterPRO feature found in the dataset?

In [29]:
# df.select("_c2").show()

In [11]:
mean_size = df.select(mean('_c2')).collect()
mean_size_float = float(str(mean_size[0]).replace('Row(avg(_c2)=', '').replace(')', ''))
mean_size_float

340.235

# Question 5
What is the top 10 most common InterPRO features?

In [28]:
interPRO_feat = []
for i in df.filter(df._c11 != '-').select('_c11').collect():
    interPRO_feat.extend(i)

c = Counter(interPRO_feat)

most_common_feats = c.most_common(10)
most_common_feats

[('IPR001697', 10),
 ('IPR005814', 6),
 ('IPR000456', 5),
 ('IPR001867', 4),
 ('IPR004358', 4),
 ('IPR003776', 3),
 ('IPR001789', 3),
 ('IPR000551', 3),
 ('IPR000515', 3),
 ('IPR036890', 3)]

# Question 6
If you select InterPRO features that are almost the same size (within 90-100%) as the protein itself, what is the top10 then?

In [ ]:
# how to get protein lenght?????
# Sequence length (e.g. 3418)